# Compare Xrefs

This notebook is to compare database cross-references, also known as xrefs, found in two ontologies or other data sources to find mappings between the ontologies. For example, given a list of data annotated to ICD10CM, find the relevant Mondo term based on the Mondo term containing an xref to the ICD10CM term.

The overall process is to first extract the xrefs from the ontologies using SPARQL queries and then find the matches. The SPARQL query can be run using the [ROBOT `query`](https://robot.obolibrary.org/query) command. ROBOT can be installed following these [instructions](https://robot.obolibrary.org/).

The Python data analysis library `pandas` will be used to find the mappings between the xrefs in each ontology. Pandas can be installed following the instructions [here](https://pandas.pydata.org/docs/getting_started/install.html).


This template notebook was created based on the [ASDC](https://www.synapse.org/Synapse:syn63102193) data set. Individual file and column names will need to be updated to match your specific data set.

## Prerequisites

A Python environment, pyenv or conda, that contains:
- pandas
- ROBOT

See installation instructions above.

## Imports and Data Preparation

In [1]:
# Imports 
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [2]:
# Get release version of mondo.owl. See https://github.com/monarch-initiative/mondo/tags for all Mondo release tags

# Comment out to prevent re-running this step while actively developing the notebook.
#!wget https://github.com/monarch-initiative/mondo/releases/download/v2025-05-06/mondo.owl -O data/v2025-05-06_mondo.owl


In [3]:
# Run query to get Mondo Xrefs for sources of interest

# Comment out to prevent re-running this step while actively developing the notebook.
#!robot query --input data/v2025-05-06_mondo.owl --use-graphs true -f tsv --query sparql/extract_mondo_xrefs.sparql reports/mondo_xrefs.tsv


In [4]:
# Read in file of Mondo xrefs

mondo_df = pd.read_csv('reports/mondo_xrefs.tsv', sep='\t')
# The `mondo_xrefs.tsv` file contains one line per mondo term per xref. Only Mondo terms with xrefs 
# are included in the file.

mondo_df.head()

,?mondo_curie,?label,?xref,?is_obsolete,?has_equivalentTo
0,MONDO:0000001,disease,ICD9:799.9,False,False
1,MONDO:0000001,disease,DOID:4,False,True
2,MONDO:0000001,disease,MEDGEN:4347,False,True
3,MONDO:0000001,disease,MESH:D004194,False,True
4,MONDO:0000001,disease,NCIT:C2991,False,True


In [5]:
mondo_df.nunique()

?mondo_curie          27108
?label                27108
?xref                131778
?is_obsolete              2
?has_equivalentTo         2
dtype: int64

In [6]:
# Read in file from other ontology source or data source

data_filepath = 'data/Down_Syndrome_Variables_ADSC_8-28-24.txt' # ADSC file
data_df = pd.read_csv(data_filepath, sep='\t', encoding='latin1')

data_df.head(10)

,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments
0,Biometric Variables,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Age at last encounter,NaN,NaN,NaN,NaN,NaN
2,NaN,Sex,NaN,NaN,NaN,NaN,NaN
3,NaN,Race,NaN,NaN,NaN,NaN,NaN
4,NaN,Ethnicity,NaN,NaN,NaN,NaN,NaN
5,NaN,Date of extraction,NaN,NaN,NaN,NaN,NaN
6,NaN,BMI,NaN,ICD10:Z68.0,NaN,NaN,NaN
7,NaN,Height at last encounter,NaN,NaN,NaN,NaN,NaN
8,NaN,Weight at last encounter,NaN,NaN,NaN,NaN,NaN
9,"Other bacterial intestinal infections, Respiratory tuberculosis",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data_df.nunique()

Variable / Field Name    161
Field Label              181
ICD-9 Codes               93
ICD-10 Codes             173
ICD-O Codes                7
bc comments                1
lk comments                0
dtype: int64

## Map terms

Find terms in data source, `data_df`, that map to Mondo term(s) based on shared xrefs between the two terms.

In [8]:
# Map the terms between each dataframe to get the ICD to Mondo translation

# Rename columns in mondo_df
mondo_clean_df = mondo_df.rename(columns={
    mondo_df.columns[0]: 'mondo_curie',
    mondo_df.columns[1]: 'mondo_label',
    mondo_df.columns[2]: 'mondo_xref',
    mondo_df.columns[3]: 'mondo_is_obsolete',
    mondo_df.columns[4]: 'mondo_has_equivalentTo'
})

display(mondo_clean_df.head())


# # Rename columns in data_df
# data_clean_df = data_df.rename(columns={
#     data_df.columns[2]: 'ds_icd9_xref',
#     data_df.columns[3]: 'ds_icd10_xref'
# })

display(data_df.head())


# Make sure xref columns have type string
mondo_clean_df['mondo_xref'] = mondo_clean_df['mondo_xref'].astype(str)
data_df['ICD-9 Codes'] = data_df['ICD-9 Codes'].astype(str)
data_df['ICD-10 Codes'] = data_df['ICD-10 Codes'].astype(str)


,mondo_curie,mondo_label,mondo_xref,mondo_is_obsolete,mondo_has_equivalentTo
0,MONDO:0000001,disease,ICD9:799.9,False,False
1,MONDO:0000001,disease,DOID:4,False,True
2,MONDO:0000001,disease,MEDGEN:4347,False,True
3,MONDO:0000001,disease,MESH:D004194,False,True
4,MONDO:0000001,disease,NCIT:C2991,False,True


,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments
0,Biometric Variables,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Age at last encounter,NaN,NaN,NaN,NaN,NaN
2,NaN,Sex,NaN,NaN,NaN,NaN,NaN
3,NaN,Race,NaN,NaN,NaN,NaN,NaN
4,NaN,Ethnicity,NaN,NaN,NaN,NaN,NaN


In [9]:
# Find ICD9 matches

icd9_merged_df = data_df.merge(
    mondo_clean_df,
    left_on='ICD-9 Codes',
    right_on='mondo_xref',
    how='left'
)

display(icd9_merged_df.head())

display(icd9_merged_df.nunique())


,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments,mondo_curie,mondo_label,mondo_xref,mondo_is_obsolete,mondo_has_equivalentTo
0,Biometric Variables,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Age at last encounter,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Sex,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Race,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Ethnicity,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Variable / Field Name     161
Field Label               181
ICD-9 Codes                94
ICD-10 Codes              174
ICD-O Codes                 7
bc comments                 1
lk comments                 0
mondo_curie                83
mondo_label                83
mondo_xref                 81
mondo_is_obsolete           1
mondo_has_equivalentTo      2
dtype: int64

In [10]:
# Find ICD10 matches

icd10_merged_df = data_df.merge(
    mondo_clean_df,
    left_on='ICD-10 Codes',
    right_on='mondo_xref',
    how='left'
)

display(icd10_merged_df.head())

display(icd10_merged_df.nunique())

,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments,mondo_curie,mondo_label,mondo_xref,mondo_is_obsolete,mondo_has_equivalentTo
0,Biometric Variables,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Age at last encounter,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Sex,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Race,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Ethnicity,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Variable / Field Name     161
Field Label               181
ICD-9 Codes                94
ICD-10 Codes              174
ICD-O Codes                 7
bc comments                 1
lk comments                 0
mondo_curie                 0
mondo_label                 0
mondo_xref                  0
mondo_is_obsolete           0
mondo_has_equivalentTo      0
dtype: int64

In [11]:
# Combine the mapping results

# Define the original data_df columns
data_columns = ['Variable / Field Name', 'Field Label', 'ICD-9 Codes', 'ICD-10 Codes', 'ICD-O Codes', 'bc comments', 'lk comments']


# Rename Mondo ICD9 xref column
icd9_selected = icd9_merged_df[
    data_columns + ['mondo_curie', 'mondo_label', 'mondo_xref', 'mondo_is_obsolete', 'mondo_has_equivalentTo']
].rename(columns={
    'mondo_xref': 'mondo_icd9_xref',
    'mondo_has_equivalentTo': 'mondo_icd9_has_equivalentTo'
})


# Rename Mondo ICD10 xref column
icd10_selected = icd10_merged_df[
    data_columns + ['mondo_curie', 'mondo_label', 'mondo_xref', 'mondo_is_obsolete', 'mondo_has_equivalentTo']
].rename(columns={
    'mondo_xref': 'mondo_icd10_xref',
    'mondo_has_equivalentTo': 'mondo_icd10_has_equivalentTo'
})


# Step 4: Merge on all original columns
combined_df = pd.merge(
    icd9_selected,
    icd10_selected,
    on=data_columns,
    how='outer'
)

# View result
combined_df.head()


display(combined_df.nunique())

Variable / Field Name           161
Field Label                     181
ICD-9 Codes                      94
ICD-10 Codes                    174
ICD-O Codes                       7
bc comments                       1
lk comments                       0
mondo_curie_x                    83
mondo_label_x                    83
mondo_icd9_xref                  81
mondo_is_obsolete_x               1
mondo_icd9_has_equivalentTo       2
mondo_curie_y                     0
mondo_label_y                     0
mondo_icd10_xref                  0
mondo_is_obsolete_y               0
mondo_icd10_has_equivalentTo      0
dtype: int64

In [12]:
# Clean up duplicate columns 

# Combine x and y columns by choosing non-null values from either column
combined_df['mondo_curie'] = combined_df['mondo_curie_x'].combine_first(combined_df['mondo_curie_y'])
combined_df['mondo_label'] = combined_df['mondo_label_x'].combine_first(combined_df['mondo_label_y'])
combined_df['mondo_is_obsolete'] = combined_df['mondo_is_obsolete_x'].combine_first(combined_df['mondo_is_obsolete_y'])

# Drop the original x and y columns if no longer needed
combined_df.drop(columns=['mondo_curie_x', 'mondo_curie_y',
                 'mondo_label_x', 'mondo_label_y',
                 'mondo_is_obsolete_x', 'mondo_is_obsolete_y'],
        inplace=True)


# View result
combined_df.head()

display(combined_df.nunique())

Variable / Field Name           161
Field Label                     181
ICD-9 Codes                      94
ICD-10 Codes                    174
ICD-O Codes                       7
bc comments                       1
lk comments                       0
mondo_icd9_xref                  81
mondo_icd9_has_equivalentTo       2
mondo_icd10_xref                  0
mondo_icd10_has_equivalentTo      0
mondo_curie                      83
mondo_label                      83
mondo_is_obsolete                 1
dtype: int64

In [13]:
# Save to file of the mapping results

combined_df.to_csv('data/adsc_icd-mondo_mappings.tsv', sep='\t', index=False)

In [14]:
# Save a file of "conditions" where there are no mappings to Mondo

display(combined_df.head())


columns_to_keep = [
    "Variable / Field Name",
    "Field Label",
    "ICD-9 Codes",
    "ICD-10 Codes",
    "ICD-O Codes",
    "bc comments",
    "lk comments"
]


no_mondo_annotations_df = combined_df[combined_df['mondo_curie'].isna()][columns_to_keep]

display(no_mondo_annotations_df.head())

,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments,mondo_icd9_xref,mondo_icd9_has_equivalentTo,mondo_icd10_xref,mondo_icd10_has_equivalentTo,mondo_curie,mondo_label,mondo_is_obsolete
0,Biometric Variables,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Age at last encounter,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Sex,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Race,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Ethnicity,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Variable / Field Name,Field Label,ICD-9 Codes,ICD-10 Codes,ICD-O Codes,bc comments,lk comments
0,Biometric Variables,NaN,nan,nan,NaN,NaN,NaN
1,NaN,Age at last encounter,nan,nan,NaN,NaN,NaN
2,NaN,Sex,nan,nan,NaN,NaN,NaN
3,NaN,Race,nan,nan,NaN,NaN,NaN
4,NaN,Ethnicity,nan,nan,NaN,NaN,NaN


In [15]:
# Save file of rows where there are no existing mappings from ICD9/10 to Mondo

no_mondo_annotations_df.to_csv("data/Down_Syndrome_Variables_ADSC_8-28-24_NO-MONDO-ANNOTATIONS.tsv", sep='\t', index=False)

In [16]:
no_mondo_annotations_df.nunique()

Variable / Field Name     72
Field Label              100
ICD-9 Codes               13
ICD-10 Codes              94
ICD-O Codes                4
bc comments                1
lk comments                0
dtype: int64